In [1]:
import pandas as pd
import numpy as np
import socket
import time

In [2]:
authorIdList = open('authorids.txt').readlines()

In [3]:
len(authorIdList)

123609

In [4]:
host = '95.217.155.82'
port = 8003

In [5]:
def get_authors_with_data(host, port, request):
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.connect((host, port))

    byt = request.encode()
    s.send(byt)
    
    answer = s.recv(1024)

    s.close()
    
    strAns = answer.decode('utf-8')
    cleanAns = clean_ans(strAns)
    
    return cleanAns

In [ ]:
authorsWData = []

#change range to len(authorIdList) or int
for i in range(100):
    author = authorIdList[i]
    request = 'aume?seco_team2_22?37\n' + author

    answer = get_authors_with_data(host, port, request)

    if answer[0] == 'No results found.':
        print(i)
        print('no results')
        if i % 1000 == 0:
            print(i)
    else:
        print(i)
        if i % 1000 == 0:
            print(i)
        authorsWData.append(author)       

textfile = open('authorsWData.txt', 'w')
for author in authorsWData:
    textfile.write(author)
textfile.close()

authorsWData

In [6]:
def make_request(host, port, request):
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.connect((host, port))

    byt = request.encode()
    s.send(byt)
    
    answer = request_loop(s)

    s.close()
    
    return answer

In [7]:
def request_loop(s, timeout=2):
    s.setblocking(0)
    total_data = []
    data = ''
    begin = time.time()

    while 1:
        if total_data and time.time()-begin > timeout:
            break
        elif time.time()-begin > timeout*2:
            break
        
        try:
            data = s.recv(4096)
            if data:
                data = data.decode('utf-8')
                total_data.append(data)
                begin = time.time()
            else:
                time.sleep(0.1)
        except:
            pass

    return ''.join(total_data)

In [8]:
def clean_ans(answer):
    answer = answer.splitlines()
    answer.pop(0)
    
    return answer

In [9]:
def create_list(answer):
    answerList = []
    if answer[0] == 'No results found.':
        return 'no'
    else:
        for i in answer:
            el = i.split('?')
            answerList.append(el)

    return answerList

In [10]:
def no_projects_per_author(answer):
    projectIdList = []
    authorDict = {}

    for m in answer:
        projectIdList.append(m[2])

    uniqueProjects = set(projectIdList)
    noProjects = len(uniqueProjects)
    
    authorDict['authorId'] = answer[0][0]
    authorDict['numberOfProjects'] = noProjects
    
    return authorDict

In [ ]:
projPerAuthor = []

for i in range(100):
    author = authorIdList[i]
    request = 'aume?seco_team2_22?37\n' + author
    
    print(i)
    
    answer = make_request(host, port, request)
    clean_answer = clean_ans(answer)
    answer_list = create_list(clean_answer)
    if answer_list != 'no':
        authProjNo = no_projects_per_author(answer_list)
        projPerAuthor.append(authProjNo)
    
projPerAuthor

In [11]:
fullList = []
for i in range(0, 80000, 100):
    callList = authorIdList[i:i+100]
    authors = ''.join(callList)
    request = 'aume?seco_team2_22?3700\n' + authors
    answer = make_request(host, port, request)
    if answer:
        cleanAns = clean_ans(answer)
        methodList = create_list(cleanAns)
        if methodList != 'no':
            for m in methodList:
                fullList.append(m)
    else:
        print('skipped ', i, i+100)
    if i % 1000 == 0:
        print(i)
        print(time.ctime())

print('Creating DataFrame...')
print(time.ctime())
authorMethodDf = pd.DataFrame(fullList, columns=['authorID', 'methodHash', 'projectID', 'projectVersion'])

print('Dropping NaN values...')
print(time.ctime())
authorMethodDf = authorMethodDf.dropna()

print('Converting columns...')
print(time.ctime())
convertDict = {
    'projectID': np.int64,
    'projectVersion': np.int64
}
authorMethodDf = authorMethodDf.astype(convertDict)

#authorMethodDf['projectID'] = pd.to_numeric(authorMethodDf['projectID'], errors='coerce')
#authorMethodDf['projectVersion'] = pd.to_numeric(authorMethodDf['projectVersion'], errors='coerce')

print('Writing to csv file...')
print(time.ctime())
authorMethodDf.to_csv('authorMethodV2.csv')

print('Done')
print(time.ctime())

0
Wed Mar 16 15:53:46 2022
1000
Wed Mar 16 15:55:10 2022
2000
Wed Mar 16 15:56:25 2022
3000
Wed Mar 16 15:57:55 2022
4000
Wed Mar 16 15:59:01 2022
skipped  4200 4300
skipped  4600 4700
5000
Wed Mar 16 15:59:44 2022
6000
Wed Mar 16 16:00:42 2022
skipped  6900 7000
7000
Wed Mar 16 16:01:32 2022
skipped  7700 7800
8000
Wed Mar 16 16:02:33 2022
skipped  8400 8500
9000
Wed Mar 16 16:03:12 2022
skipped  9200 9300
10000
Wed Mar 16 16:04:02 2022
skipped  10900 11000
11000
Wed Mar 16 16:04:54 2022
12000
Wed Mar 16 16:05:43 2022
13000
Wed Mar 16 16:06:41 2022
skipped  13800 13900
14000
Wed Mar 16 16:07:30 2022
skipped  14600 14700
15000
Wed Mar 16 16:08:24 2022
16000
Wed Mar 16 16:09:16 2022
skipped  16300 16400
17000
Wed Mar 16 16:10:10 2022
18000
Wed Mar 16 16:11:15 2022
19000
Wed Mar 16 16:12:14 2022
20000
Wed Mar 16 16:13:00 2022
21000
Wed Mar 16 16:13:50 2022
skipped  21200 21300
22000
Wed Mar 16 16:14:39 2022
skipped  22500 22600
23000
Wed Mar 16 16:15:38 2022
24000
Wed Mar 16 16:16:30 202

In [12]:
authorMethodDf

,authorID,methodHash,projectID,projectVersion
0,e157cee2-b5b5-2466-14a8-717e9de1ce4c,b8a899ee5f7c93361e215edf5be05462,3477431694,1273494526000
1,a0c11df7-5118-e7d6-59c3-65223bf5b5a0,64132196448efd90223a4f7f16f0ae25,1821731652,1275264487000
2,a0c11df7-5118-e7d6-59c3-65223bf5b5a0,b8a76bcb30cd6afc1476ad39fc355317,949829554,1275264520000
3,a0c11df7-5118-e7d6-59c3-65223bf5b5a0,165944a511b4539a285f269aaac95437,2915245721,1272340059000
4,a0c11df7-5118-e7d6-59c3-65223bf5b5a0,165944a511b4539a285f269aaac95437,2915245721,1278425588000
...,...,...,...,...
30250261,1ce151e9-89e4-c24e-19f4-a2528a2da121,431f4e8f211d5f9a56fae1a0e8d47320,352558408,1426521636
30250262,1ce151e9-89e4-c24e-19f4-a2528a2da121,0ad0aaa9d2d046bfe7dc3aafc5f3566e,352558408,1443456392
30250263,1ce151e9-89e4-c24e-19f4-a2528a2da121,0ad0aaa9d2d046bfe7dc3aafc5f3566e,352558408,1444969846
30250264,edf67142-19da-8f47-ab69-6c1b37b94010,e606abc81be49f08071fc04e3386ad47,2011939588,1400533413


In [13]:
authorMethodDf.dtypes

authorID          object
methodHash        object
projectID          int64
projectVersion     int64
dtype: object

In [ ]:
callList = authorIdList[850:900]
authors = ''.join(callList)
request = 'aume?seco_team2_22?1850\n' + authors
answer = make_request(host, port, request)
print(answer)
#cleanAns = clean_ans(answer)
#methodList = create_list(cleanAns)

In [ ]:
for i in range(850,900,1):
    author = authorIdList[i]
    request = 'aume?seco_team2_22?37\n' + author
    answer = make_request(host,port,request)
    print(answer)

In [ ]:
author = authorIdList[4]
request = 'aume?seco_team2_22?37\n' + author

answer = make_request(host, port, request)
answer

In [ ]:
clean_answer = clean_ans(answer)
clean_answer

In [ ]:
answer_list = create_list(clean_answer)
answer_list

In [ ]:
authProjNo = no_projects_per_author(answer_list)
authProjNo

In [ ]:
request = 'aume?seco_team2_22?111\n' + authorIdList[3435] + authorIdList[876] + authorIdList[946]

In [ ]:
answer = make_request(host, port, request)
cleanAns = clean_ans(answer)
methList = create_list(cleanAns)
methList

In [ ]:
author = authorIdList[4890]
request = 'idau?seco_team2_22?37\n' + author

author

In [ ]:
devs = []

for i in range(50):
    author = authorIdList[i]
    request = 'idau?seco_team2_22?37\n' + author
    
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.connect((host, port))

    byt = request.encode()
    s.send(byt)
    answer = s.recv(4096)

    s.close()

    answer = answer.decode('utf-8')
    answer = answer.splitlines()
    devList = answer[1].split('?')
    devs.append(devList)

devs

In [ ]:
df = pd.DataFrame.from_records(devs)
df

In [ ]:
df.to_csv('authors.csv')

In [ ]:
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.connect((host, port))

byt = request.encode()
s.send(byt)
answer = s.recv(4096)

s.close()

answer = answer.decode('utf-8')
answer = answer.splitlines()
devList = answer[1].split('?')

devList

In [ ]:
author = authorIdList[17]
request = 'aume?seco_team2_22?37\n' + author

In [ ]:
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.connect((host, port))

byt = request.encode()
s.send(byt)

def request_loop_old(s, timeout=2):
    s.setblocking(0)
    total_data = []
    data = ''
    begin = time.time()

    while 1:
        if total_data and time.time()-begin > timeout:
            break
        elif time.time()-begin > timeout*2:
            break
        
        try:
            data = s.recv(4096)
            if data:
                data = data.decode('utf-8')
                total_data.append(data)
                begin = time.time()
            else:
                time.sleep(0.1)
        except:
            pass

    return ''.join(total_data)

answer = request_loop(s)

s.close()

answer

In [ ]:
answer = answer.decode('utf-8')
answer = answer.splitlines()
answer

In [ ]:
answer.pop(0)
answer

In [ ]:
if answer[0] == 'No results found.':
    print('No results')
else:
    answerList = []

    for i in answer:
        method = i.split('?')
        answerList.append(method)

    projectIdList = []

    for m in answerList:
        projectIdList.append(m[2])

    uniqueProjects = set(projectIdList)
    print(len(uniqueProjects))